## Cashflow management problem

__Details of the problem are as follows:__
+ __Periods:__ Jan-Jun (Cash Flows pay the 1st each month)
+ __Liability__ = [150.0,100.0,-200.0,200.0,-50.0,-350.0]
+ __Financial Instruments:__
    + __Line of credit:__ Limit 100K, interest = 1% per month
    + __Commercial paper:__ Unlimited, duration = 3 months, interest = 2% for 3 months
    + __Bank account:__ Unlimited, interest = 0.3% per month

In [15]:
import numpy as np
from tabulate import *
import cvxpy as cvx #library to solve optimization problems

__Set up the data and variables for the problem__

In [37]:
#Creates array for CFs in a row vector
liability = np.array([150.0,100.0,-200.0,200.0,-50.0,-350.0])

#Variables: 1) time period #s (cols), 2) LoC%, 3) paper%, 4) rfr%, 5) paper period, 6) LoC upper limit
ntimes = len(liability); #this returns a value of 6 AND NOT 5 as np.array is counted in full unlike list
loan_int = 1;
paper_int = 2;
rf_int = 0.3;
paper_period = 3;
loan_limit = 100;

__Set up the ("decision") variables__ 

In [38]:
#int obj for cvx.Variables are 5/3/6 for x/y/z and LoC/paper/cash, respectively (diff time constraints)
x = cvx.Variable(ntimes-1)
y = cvx.Variable(ntimes-paper_period)
z = cvx.Variable(ntimes)

__Set up the constraints for each time period and the objective function__

In [44]:
#creates empty list to hold each constraint as we do our loop thru periods below 
constraints = [];

#create flow constraints for each time period (loops thru n-periods using range of periods)
for t in range(ntimes):    
    
    #creates empty list to be filled with decision variables thru loop (sums CFs for period)
    expr = []

    #if current month greater than 1st month: subtract LoC interest calc'd on last month's balance
    if (t>0):
        expr += [-(1+loan_int/100)*x[t-1]]; 
    #if current month less than final month: add LoC loans taken this month
    if (t<ntimes-1):
        expr += [x[t]] 
    #if current month greater than 3rd month: subtract paper interest calc'd -3 month's ago balance
    if (t>paper_period-1):
        expr += [-(1+paper_int/100)*y[t-paper_period]]; 
    #if current month is less than 4th month: add paper loans taken this month
    if (t<ntimes-paper_period):
        expr += [y[t]]; 
    #if current month greater than 1st month: add cash return calc'd on last month's cash balance
    if (t>0):
        expr += [(1+rf_int/100)*z[t-1]];     
    #add current month CF to calc (flip sign)
    expr += [-z[t]];     
    
    #create object for t period in loop so cash >= liability and add it to constraint list
    constraints += [sum(expr) >= liability[t]]; 

#add additional constraint of LoC's upper bound to constraint list
constraints += [x <= loan_limit]

#add non-negativity constraint for decision variables (financial tools) to constraint list
constraints += [x>=0, y>=0, z>=0];

#objective function (ntimes-1 b/c ntimes is max(range index)+1 given it's a length of an np.array)
objective = cvx.Maximize(z[ntimes-1]);

__Set up the optimization and solve it using the default cvxpy solver__

In [71]:
#creates/solves problem
prob = cvx.Problem(objective, constraints)
prob.solve()

#sets print options
np.set_printoptions(precision=4,suppress=True)

#prints values
print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Problem value: %.2f' % prob.value);
    print('Line of Credit values: ' + str(np.round(x.value.transpose(), 2)))
    print('Commercial Paper values: ' + str(np.round(y.value.transpose(), 2)))
    print('Cash values: ' + str(np.round(z.value.transpose(), 2)))

Problem status: optimal
Problem value: 142.50
Line of Credit values: [ 0.   12.96  0.    0.   38.78]
Commercial Paper values: [150.    87.04 165.03]
Cash values: [  0.     0.   351.94   0.     0.   142.5 ]


### You left off here so everything below you need to study/review!!!

In [74]:
#prints all possible types of solvers included in cvxpy package
print(cvx.installed_solvers()) 

['CBC', 'CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLOP', 'GLPK', 'GLPK_MI', 'GUROBI', 'MOSEK', 'OSQP', 'PDLP', 'SCIP', 'SCIPY', 'SCS', 'XPRESS']


__Solve the same LP using GLPK_MI__

In [73]:
#creates/solves problem using GLPK.MI solver
prob_glpk = cvx.Problem(objective, constraints)
prob_glpk.solve(solver=cvx.GLPK_MI)

#sets print options
np.set_printoptions(precision=2,suppress=True)

#prints values
print('Problem (GLPK) status: ' + str(prob_glpk.status));
if (prob_glpk.status == 'optimal'):
    print('Problem (GLPK) value: %.2f' % prob_glpk.value);
    print('Line of Credit (GLPK) values: ' + str(np.round(x.value.transpose(), 2)))
    print('Commercial Paper (GLPK) values: ' + str(np.round(y.value.transpose(), 2)))
    print('Cash values (GLPK): ' + str(np.round(z.value.transpose(), 2)))

Problem (GLPK) status: optimal
Problem (GLPK) value: 142.50
Line of Credit (GLPK) values: [-0.   50.98 -0.   -0.   -0.  ]
Commercial Paper (GLPK) values: [150.    49.02 203.43]
Cash values (GLPK): [ -0.    -0.   351.94  -0.    -0.   142.5 ]


### Sensitivity analysis via basis

In [10]:
# Formulate a standard form LP
n = (ntimes-1) + (ntimes-paper_period) + ntimes 
m = ntimes

xoffset = 0
yoffset = xoffset + (ntimes-1) 
zoffset = yoffset + (ntimes-paper_period) 

A = np.zeros((m,n));
# liability constraints
for i in range(ntimes):
    a = np.zeros(n)
    if (i<ntimes-1):
        a[i] = 1; # loan available
    if (i > 0):
        a[i-1] = -(1+loan_int/100) # loan has to repaid
    if (i < ntimes-paper_period):
        a[yoffset+i] = 1 # paper can be issued 
    if (i >= paper_period):
        a[yoffset + i - paper_period] = -(1+paper_int/100)
    if (i > 0):
        a[zoffset + i-1] = (1+rf_int/100)
    a[zoffset + i] = -1
    A[i,:] = a
    
b = np.array(liability).T
c = np.zeros((n,1))
c[n-1] = 1
    

In [11]:
# Define the basis from the gurobi solution
tol = 1e-6
B = [i for i in range(ntimes-1) if x.value[i] > tol] 
B += [(yoffset + i) for i in range(ntimes-paper_period) if y.value[i]>tol]
B += [(zoffset + i) for i in range(ntimes) if z.value[i]>tol]
B = np.array(B)
print(B)

[ 1  5  6  7 10 13]


### Sensititive analysis with respect to the right handside

In [12]:
# define A_B^{-1}
invA_B = np.linalg.inv(A[:,B])

# basic solution 
xB = invA_B.dot(b);

LB = -np.inf*np.ones((m,1))
UB = np.inf*np.ones((m,1))

for i in range(m):
    ei = np.zeros((m,1)); ei[i] = 1;
    d = invA_B.dot(ei);
    for j in range(m):
        if (float(d[j]>0)):
            LB[i] = max(LB[i], -float(xB[j])/float(d[j]))
        elif (float(d[j]<0)):
            UB[i] = min(UB[i], -float(xB[j])/float(d[j]))
    LB[i] += b[i]
    UB[i] += b[i]
    
np.set_printoptions(precision=4,suppress=True)
p = (((c[B]).T).dot(invA_B)).T

table = [[t+1, p[t], LB[t], b[t], UB[t]] for t in range(m)];
print(tabulate(table, headers=["Time","Shadow price", "RHS Low", "RHS", "RHS Up"], floatfmt=".4f")); 

  Time    Shadow price    RHS Low        RHS     RHS Up
------  --------------  ---------  ---------  ---------
     1         -1.0373    -0.0000   150.0000   287.3745
     2         -1.0302    49.0196   100.0000   238.3197
     3         -1.0200  -403.4344  -200.0000   -60.2971
     4         -1.0169    -4.0447   200.0000   340.1220
     5         -1.0100  -102.0000   -50.0000     0.0000
     6         -1.0000  -inf       -350.0000  -207.5031


### What is maximum interest rate would one still move liability from time 1 to time 6? And how much?

The first part, i.e at what maximum interest rate would it still be profitable to move liability, can be answered using the shadow prices. 
+ An infinitesimal decrease $\delta$ in the liablility at time $t=1$ increases the objective by $-p_0 \delta$,  
+ and the increase $(1+r)\delta$ at time $t=6$ decreases the objective by $(1+r)p_5\delta$
+ Move profitable if 
$$
-p_0 + (1+r)p_5 \geq 0 \quad \Rightarrow \quad r \leq \frac{p_0}{p_5} - 1
$$

However, the second part of the question, i.e. how much would one move, cannot be accurately using the information in the above table, because the table was created by assuming that only one component of the $b$ vector changes at a time. (Or, can it?)

So, what happens when multiple components change? One can apply the same methodology. Suppose
$$
b = b + \theta d
$$
for some $\theta \in \mathbb{R}$. The current basis will remain optimal provided
$$
x_B = A_B^{-1}b + \theta A_{B}^{-1} d \geq 0.
$$
One can use this inequality to compute upper and lower bounds on $\theta$. 

In this particular case
$$
d = \begin{bmatrix} -1 \\ 0 \\ \vdots \\ 0 \\ (1+r) \end{bmatrix}
$$

In [13]:
month = 6;
rmax = (p[0]/p[month-1]-1)*100;
print('Maximum interest rate rmax = %0.3f%%' % (rmax))

# basic solution 
xB = invA_B.dot(b);

theta_low = -np.inf
theta_up = np.inf

d = np.zeros((m,1)); d[0] = -1; d[month-1] = (1+rmax/100); 
d = invA_B.dot(d);
for j in range(m):
    if (float(d[j]>0)):
        theta_low = max(theta_low, -float(xB[j])/float(d[j]))
    elif (float(d[j]<0)):
        theta_up = min(theta_up, -float(xB[j])/float(d[j]))
print('Maximum liability moved = %.3f' % theta_up)

Maximum interest rate rmax = 3.729%
Maximum liability moved = 150.000
